# Quickstart

In this tutorial we do the following :

1. __Examine the__ ``'network'`` __parameter tree__
    1. Models (``neuron_models``, ``recorder_models``, ``synapse_models``)
    2. Layers (``layers``)
    3. Projections (``projection_models`` and ``topology`` subtrees
    5. Population and projection recorders (``recorders``)


2. __Examine the full simulation parameter tree__
    1. ``network`` params subtree
    2. ``kernel`` params subtree
    3. ``session_models`` params subtree
    4. ``simulation`` params subtree


3. __Run a full simulation__
    1. Load parameter tree from one or multiple files
    1. Using the ``Simulation`` object
    2. Using the ``denest.run()`` method
    3. From the command line


In [1]:
import nest
import yaml
from pathlib import Path
from pprint import pprint

from denest import *
import denest

In [2]:
PARAMS_DIR = Path('./data/params')
OUTPUT_DIR = Path('./data/outputs/output')

## ``"network"`` tree

A full NEST network is represented in deNEST by the ``Network`` object.

The `Network` object is initialized from the full ``"network"`` parameter tree.
The `'network'` tree should have specific children subtree, each used to initialize different elements of the network.

In [3]:
net_tree = ParamsTree.read(PARAMS_DIR/'network_tree.yml').children['network']

In [4]:
# Full 'network' tree
net_tree

ParamsTree(name='network', parent='None')
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
    my_neuron:
      params:
        nest_model: ht_neuron
      nest_params:
        g_KL: 1.0
        g_NaL: 1.0
      l1_exc:
        params: {}
        nest_params:
          V_m: -44.0
  
  ... [116 lines] ...

      - layers:
        - input_layer
        populations: null
        model: my_spike_detector
      projection_recorders:
      - source_layers:
        - l1
        source_population: l1_exc
        target_layers:
        - l1
        target_population: l1_inh
        projection_model: proj_1_AMPA
        model: weight_recorder
    nest_params: {}
  

In [5]:
print("'network' tree's expected subtrees:", net_tree.children.keys())

'network' tree's expected subtrees: dict_keys(['neuron_models', 'synapse_models', 'layers', 'projection_models', 'topology', 'recorder_models', 'recorders'])


### Models ( ``'neuron_model'``, ``'synapse_models'``, ``'recorder_models'`` subtrees)

We can define neuron, recorder, stimulator and synapse models with arbitrary parameters from parameter trees.

Each leaf corresponds to a new (named) model. Its ``nest_params`` and ``params`` are hierarchically inherited.

The ``nest_model`` used is specified in the leaf's ``params``

#### ``'neuron_models'`` tree

In [6]:
net_tree.children['neuron_models']

ParamsTree(name='neuron_models', parent='network')
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
  

Below are the leaves of the tree. Each initializes a model. The `nest_params` and `params` are inherited from ancestor leaves.

In [7]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in net_tree.children['neuron_models'].leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: l1_exc)',
  "leaf `params`: {'nest_model': 'ht_neuron'}",
  "leaf `nest_params`: {'g_KL': 1.0, 'g_NaL': 1.0, 'V_m': -44.0}"),
 ('leaf name: l1_inh)',
  "leaf `params`: {'nest_model': 'ht_neuron'}",
  "leaf `nest_params`: {'g_KL': 1.0, 'g_NaL': 1.0, 'V_m': -55.0}")]

#### ``'recorder_models'`` subtree

Same thing as for neuron models:

In [8]:
net_tree.children['recorder_models']

ParamsTree(name='recorder_models', parent='network')
  params: {}
  nest_params:
    record_to:
    - memory
    - file
  weight_recorder:
    params:
      nest_model: weight_recorder
    nest_params: {}
  my_multimeter:
    params:
      nest_model: multimeter
    nest_params:
      record_from:
      - V_m
  my_spike_detector:
    params:
      nest_model: spike_detector
    nest_params: {}
  

In [9]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in net_tree.children['recorder_models'].leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: weight_recorder)',
  "leaf `params`: {'nest_model': 'weight_recorder'}",
  "leaf `nest_params`: {'record_to': ['memory', 'file']}"),
 ('leaf name: my_multimeter)',
  "leaf `params`: {'nest_model': 'multimeter'}",
  "leaf `nest_params`: {'record_to': ['memory', 'file'], 'record_from': ['V_m']}"),
 ('leaf name: my_spike_detector)',
  "leaf `params`: {'nest_model': 'spike_detector'}",
  "leaf `nest_params`: {'record_to': ['memory', 'file']}")]

#### ``'synapse_model'`` subtree

- Same thing as for neuron models, with as a bonus a convenient way of specifying the receptor type of the synapse
- If specifying the ``receptor_type`` and ``target_model`` in the ``SynapseModel`` params, the corresponding port is determined automatically
- Connection __'weight'__ nest_param should be specified in the projections parameters rather than the synapse parameters

In [10]:
pprint(net_tree.children['synapse_models'])

ParamsTree(name='synapse_models', parent='network')
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
  


In [11]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in net_tree.children['synapse_models'].leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: my_AMPA_synapse)',
  "leaf `params`: {'nest_model': 'ht_synapse', 'receptor_type': 'AMPA', 'target_neuron': 'ht_neuron'}",
  'leaf `nest_params`: {}'),
 ('leaf name: my_GABAA_synapse)',
  "leaf `params`: {'nest_model': 'ht_synapse', 'receptor_type': 'GABA_A', 'target_neuron': 'ht_neuron'}",
  'leaf `nest_params`: {}')]

### Layers ( `'layers'` subtree)

- As for models, we can create ``nest.Topology`` layers from the leaves of a tree.
- The elements can be nest models with their default parameters, or the ones we just created with custom params
- For layers of stimulator devices, we can use the ``InputLayer`` object, which can automatically create paired parrot neurons for each stimulator units, by adding ``type: 'InputLayer'`` to the params
- The leaves should have a ``population`` entry in the ``params`` field, that specifies the number of units of each model at each layer location (replaces the ``elements`` nest_params)

In [12]:
layer_tree = net_tree.children['layers']
layer_tree

ParamsTree(name='layers', parent='network')
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        add_parrots: true
        populations:
          spike_generator: 1
      nest_params: {}
    l1:
      params:
        populations:
          l1_exc: 4
          l1_inh: 2
      nest_params: {}
  

In [13]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in layer_tree.leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: input_layer)',
  "leaf `params`: {'type': 'InputLayer', 'add_parrots': True, 'populations': {'spike_generator': 1}}",
  "leaf `nest_params`: {'rows': 5, 'columns': 5, 'extent': [5.0, 5.0], 'edge_wrap': True}"),
 ('leaf name: l1)',
  "leaf `params`: {'type': None, 'populations': {'l1_exc': 4, 'l1_inh': 2}}",
  "leaf `nest_params`: {'rows': 5, 'columns': 5, 'extent': [5.0, 5.0], 'edge_wrap': True}")]

### Projections (`'projection_models'` and `'topology'` subtrees)

We create projections using a two step process:

1. Create ``ProjectionModel`` objects from a tree. Each named leaf will define a template from which individual projections can inherit their parameters (``'projection_models`` subtree)


2. Create ``Projection`` objects from a list, specifying for each item the source layer x population, target layer x population and the projection model to inherit parameters from (``'topology'`` subtree)

#### Define templates from the `projection_models` tree

In [14]:
proj_model_tree = net_tree.children['projection_models']
proj_model_tree

ParamsTree(name='projection_models', parent='network')
  params: {}
  nest_params:
    connection_type: divergent
    mask:
      circular:
        radius: 2.0
    kernel: 1.0
  proj_1_AMPA:
    params: {}
    nest_params:
      synapse_model: my_AMPA_synapse
      weights: 1.0
  proj_2_GABAA:
    params: {}
    nest_params:
      synapse_model: my_GABAA_synapse
      weights: 2.0
  

In [15]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in net_tree.children['projection_models'].leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: proj_1_AMPA)',
  'leaf `params`: {}',
  "leaf `nest_params`: {'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0, 'synapse_model': 'my_AMPA_synapse', 'weights': 1.0}"),
 ('leaf name: proj_2_GABAA)',
  'leaf `params`: {}',
  "leaf `nest_params`: {'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0, 'synapse_model': 'my_GABAA_synapse', 'weights': 2.0}")]

#### Define individual projections from the `topology` tree

The list of projections is defined in the `projections` params of the `topology` tree

Check out the doc of `Network.build_projections` for expected formatting

For each item in the list, a projection is created for each of the `<source_layer> x <target_layer>` combinations.
The `params` and `nest_params` are inherited from the template `projection_model`

In [16]:
topo_tree = net_tree.children['topology']
topo_tree

ParamsTree(name='topology', parent='network')
  params:
    projections:
    - source_layers:
      - input_layer
      source_population: parrot_neuron
      target_layers:
      - l1
      target_population: l1_exc
      projection_model: proj_1_AMPA
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      projection_model: proj_1_AMPA
    - source_layers:
      - l1
      source_population: l1_inh
      target_layers:
      - l1
      target_population: l1_exc
      projection_model: proj_2_GABAA
  nest_params: {}
  

### Recorders (`recorders` subtree)

Similarly to the `topology` tree, recorders are defined from lists.

We separate recorders connected to synapses (eg weight recorder) and those connected to units (eg spike detectors),
which are defined in the `projection_recorders` and `population_recorders` params (resp.) of the `recorders` tree.

Check out the doc of the ``Network.build_recorders``, ``Network.build_population_recorders`` and
``Network.build_projection_recorders`` methods for expected formatting

The parameters of the recorders can be changed by using custom recorder models (in the `recorder_models` tree, see above) 

In [17]:
recorders_tree = net_tree.children['recorders']
recorders_tree

ParamsTree(name='recorders', parent='network')
  params:
    population_recorders:
    - layers:
      - l1
      populations:
      - l1_exc
      model: my_multimeter
    - layers:
      - input_layer
      populations: null
      model: my_spike_detector
    projection_recorders:
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      projection_model: proj_1_AMPA
      model: weight_recorder
  nest_params: {}
  

## Full parameter tree


A full simulation consists in

- kernel initialization
- network creation
- running the network in multiple 'sessions' at the start of which some modifications to the network can be made

The full simulation is represented by the `Simulation` object, which is initialized from a parameter tree with the following subtrees:

1. `'network'` subtree, passed to `Network()`

1. `'kernel'` subtree, passed to `Simulation.init_kernel()`

1. `'session_models'` subtree, defining the parameters for each individual `Session`

1. `'simulation'` subtree, containing the list of sessions that we run and the input/output directory paths

In [18]:
full_tree = ParamsTree.read(PARAMS_DIR/'parameter_tree.yml')

In [19]:
full_tree.children.keys()

dict_keys(['kernel', 'simulation', 'session_models', 'network'])

### `'network'` subtree

The `'network'` subtree is expected to have the following children, as we saw above:

In [20]:
net_tree = full_tree.children['network']
net_tree.children.keys()

dict_keys(['neuron_models', 'synapse_models', 'layers', 'projection_models', 'topology', 'recorder_models', 'recorders'])

It is used to initialize a `Network` object, representing the full NEST network.

In [21]:
net = Network(net_tree)

2020-06-30 13:51:20,459 [denest.network] INFO: Build N=2 ``Model`` objects
2020-06-30 13:51:20,460 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-06-30 13:51:20,461 [denest.network] INFO: Build N=3 ``Model`` objects
2020-06-30 13:51:20,467 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-06-30 13:51:20,469 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-30 13:51:20,470 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-30 13:51:20,471 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects
2020-06-30 13:51:20,476 [denest.network] INFO: Build N=3 ``TopoProjection`` objects
2020-06-30 13:51:20,484 [denest.network] INFO: Build N=2 population recorders.
2020-06-30 13:51:20,485 [denest.network] INFO: Build N=1 projection recorders.


### `'kernel'` subtree

The kernel subtree is passed to the `Simulation.init_kernel` method.

- All `nest_params` are passed to nest.SetKernelStatus
- Two optional `params`:
    - `nest_seed` (used to initialize at once the thread-wide and general nest rsg. The `msd`, `grng_seed` and `rng_seed` nest params are reserved)
    - `extension_modules` (used to call `nest.Install()`)
- Note the `data_path` parameter is set automatically from the simulation's `'output_dir'`

In [22]:
full_tree.children['kernel']

ParamsTree(name='kernel', parent='None')
  params:
    nest_seed: 10
    extension_modules: []
  nest_params:
    resolution: 0.5
    overwrite_files: true
  

### `'session_models'` subtree

The leaves of the `'session_models'` subtree define templates from which the parameters of each `Session` object can be inherited.

The following `params` are recognized:

- ``simulation_time`` (float): Duration of the session in ms.
    (mandatory)
- ``reset_network`` (bool): If true, ``nest.ResetNetwork()`` is
    called during session initialization (default ``False``)
- ``record`` (bool): If false, the ``start_time`` field of
    recorder nodes in NEST is set to the end time of the
    session, so that no data is recorded during the session
    (default ``True``)
- ``shift_origin`` (bool): If True, the ``origin`` flag of the
    stimulation devices of all the network's ``InputLayer``
    layers is set to the start of the session during
    initialization. Useful to repeat sessions when the
    stimulators are eg spike generators.
- ``unit_changes`` (list): List describing the changes applied
    to certain units before the start of the session.
    Passed to ``Network.set_state``.
- ``synapse_changes`` (list): List describing the changes
    applied to certain synapses before the start of the session.
    Passed to ``Network.set_state``. Refer to that
    method for a description of how ``synapse_changes`` is
    formatted and interpreted. No changes happen if empty.
    (default [])

In [23]:
full_tree.children['session_models']

ParamsTree(name='session_models', parent='None')
  params:
    record: true
    shift_origin: true
    simulation_time: 100.0
  nest_params: {}
  warmup:
    params:
      record: false
    nest_params: {}
  2_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
  
  ... [3 lines] ...

          - 10.0
    nest_params: {}
  3_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
        nest_params:
          spike_times:
          - 1.0
          - 10.0
          - 20.0
    nest_params: {}
  

In [24]:
print("Tree's leaves and their inherited parameters:")
[
    (f'leaf name: {l.name})',
      f'leaf `params`: {dict(l.params)}',
      f'leaf `nest_params`: {dict(l.nest_params)}')
     for l in full_tree.children['session_models'].leaves()
]

Tree's leaves and their inherited parameters:


[('leaf name: warmup)',
  "leaf `params`: {'record': False, 'shift_origin': True, 'simulation_time': 100.0}",
  'leaf `nest_params`: {}'),
 ('leaf name: 2_spikes)',
  "leaf `params`: {'record': True, 'shift_origin': True, 'simulation_time': 100.0, 'unit_changes': [{'layers': ['input_layer'], 'population_name': 'spike_generator', 'nest_params': {'spike_times': [1.0, 10.0]}}]}",
  'leaf `nest_params`: {}'),
 ('leaf name: 3_spikes)',
  "leaf `params`: {'record': True, 'shift_origin': True, 'simulation_time': 100.0, 'unit_changes': [{'layers': ['input_layer'], 'population_name': 'spike_generator', 'nest_params': {'spike_times': [1.0, 10.0, 20.0]}}]}",
  'leaf `nest_params`: {}')]

Check out the ``set_network_state`` tutorial or the documentation of the ``Network.set_state()`` method for details on how to modify the state of the network at the start of a simulation

### `'simulation'` subtree

The 'simulation' subtree contains the following parameters:

- ``output_dir`` (str): Path to the output directory
    (default 'output').
- ``input_dir`` (str): Path to the directory in which
    input files are searched for for each session.
    (default 'input')
- ``sessions`` (list(str)): Order in which sessions are
    run. Elements of the list should be the name of
    session models defined in the ``session_models``
    parameter subtree (default [])

In [25]:
full_tree.children['simulation']

ParamsTree(name='simulation', parent='None')
  params:
    output_dir: data/outputs/output
    sessions:
    - warmup
    - 3_spikes
    - 2_spikes
    - 3_spikes
  nest_params: {}
  

## Run a full simulation

### Load the full parameter tree

The full parameter tree can be loaded from a single file, or from multiple files which are merged.

**From a single file** by using the `yaml.load`  or the `ParamsTree.read` methods:

In [26]:
ParamsTree.read(PARAMS_DIR/'parameter_tree.yml')

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data/outputs/output
      sessions:
      - warmup
      - 3_spikes
  
  ... [169 lines] ...

        - layers:
          - input_layer
          populations: null
          model: my_spike_detector
        projection_recorders:
        - source_layers:
          - l1
          source_population: l1_exc
          target_layers:
          - l1
          target_population: l1_inh
          projection_model: proj_1_AMPA
          model: weight_recorder
      nest_params: {}
  

**By merging multiple files**, using the `denest.load_trees` method:

In [27]:
!cat {PARAMS_DIR/'tree_paths.yml'}

- './network_tree.yml'
- './simulation.yml'
- './session_models.yml'
- './kernel.yml'


In [28]:
full_tree = load_trees(PARAMS_DIR/'tree_paths.yml')

2020-06-30 13:51:21,241 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:51:21,245 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:51:21,260 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']


In [29]:
full_tree

ParamsTree(name='data/params/tree_paths.yml', parent=None)
  params: {}
  nest_params: {}
  session_models:
    params:
      record: true
      shift_origin: true
      simulation_time: 100.0
    nest_params: {}
    3_spikes:
      params:
        unit_changes:
        - layers:
          - input_layer
          population_name: spike_generator
          nest_params:
  
  ... [168 lines] ...

          params:
            populations:
              l1_exc: 4
              l1_inh: 2
          nest_params: {}
  simulation:
    params:
      output_dir: ./data/outputs/output
      sessions:
      - warmup
      - 3_spikes
      - 2_spikes
      - 3_spikes
    nest_params: {}
  

### Run with the `Simulation` object

We load the full parameter tree:

In [30]:
tree = ParamsTree.read(PARAMS_DIR/'parameter_tree.yml')

And then initialize the `Simulation`:

In [31]:
sim = Simulation(tree, output_dir=None, input_dir=None)

2020-06-30 13:51:21,596 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-30 13:51:21,598 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-30 13:51:21,600 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-30 13:51:21,609 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-30 13:51:21,609 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-30 13:51:21,614 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'data/outputs/output/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-30 13:51:21,618 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-30 13:51:21,636 [denest.simulation] INFO:   Installing external modules...
2020-06-30 13:51:21,637 [denest.simulation] INFO:   Finished installing external modules
2020-06-30 13:51:21,645 [denest.simul

We can inspect the simulation elements:

In [32]:
sim.network

Network(params: {}
nest_params: {}
neuron_models:
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
synapse_models:
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
layers:
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        add_parrots: true
        populations:
          spike_generator: 1
      nest_params: {}
    l1:
      para

In [33]:
sim.network.layers

{'input_layer': InputLayer(input_layer, 
  {'type': 'InputLayer',
   'add_parrots': True,
   'populations': {'spike_generator': 1, 'parrot_neuron': 1}}{'rows': 5,
   'columns': 5,
   'extent': [5.0, 5.0],
   'edge_wrap': True,
   'elements': ('spike_generator', 1, 'parrot_neuron', 1)}),
 'l1': Layer(l1, 
  {'type': None, 'populations': {'l1_exc': 4, 'l1_inh': 2}}{'rows': 5,
   'columns': 5,
   'extent': [5.0, 5.0],
   'edge_wrap': True,
   'elements': ('l1_exc', 4, 'l1_inh', 2)})}

In [34]:
sim.sessions

[Session(00_warmup, {'record': False,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': []}),
 Session(01_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0, 20.0]},
                    'population_name': 'spike_generator'}]}),
 Session(02_2_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0]},
                    'population_name': 'spike_generator'}]}),
 Session(03_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
        

Finally, we actually run all the sessions in NEST by calling `Simulation.run()`:

In [35]:
sim.run()

2020-06-30 13:51:22,828 [denest.simulation] INFO: Running 4 sessions...
2020-06-30 13:51:22,831 [denest.simulation] INFO: Running session: '00_warmup'...
2020-06-30 13:51:22,838 [denest.session] INFO: Initializing session...
2020-06-30 13:51:22,840 [denest.network.recorders] INFO:   Setting status for recorder my_multimeter_l1_l1_exc: {'start': 100.0}
2020-06-30 13:51:22,843 [denest.network.recorders] INFO:   Setting status for recorder my_spike_detector_input_layer_parrot_neuron: {'start': 100.0}
2020-06-30 13:51:22,846 [denest.network.recorders] INFO:   Setting status for recorder weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh: {'start': 100.0}
2020-06-30 13:51:22,847 [denest.session] INFO: Setting `origin` flag to `0.0` for all stimulation devices in ``InputLayers`` for session `00_warmup`
2020-06-30 13:51:22,863 [denest.session] INFO: Finished initializing session

2020-06-30 13:51:22,864 [denest.session] INFO: Running session '00_warmup' for 100 ms
2020-06-30 13:51:23,077 [denest

In [36]:
!ls {sim.output_dir}

data               parameter_tree.yml session_times.yml  versions.txt


In [37]:
!ls {Path(sim.output_dir)/'data'}

my_multimeter_l1_l1_exc-203-0.dat
my_multimeter_l1_l1_exc.yml
my_spike_detector_input_layer_parrot_neuron-204-0.gdf
my_spike_detector_input_layer_parrot_neuron.yml
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh-205-0.csv
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh.yml


### Run using the `denest.run` function

The `run` function is a shorthand for the above steps:

In [38]:
denest.run(PARAMS_DIR/'tree_paths.yml')

2020-06-30 13:51:25,053 [denest] INFO: 

=== RUNNING SIMULATION ========================================================

2020-06-30 13:51:25,055 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:51:25,061 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:51:25,063 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-30 13:51:25,119 [denest] INFO: Initializing simulation...
2020-06-30 13:51:25,146 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-30 13:51:25,149 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-30 13:51:25,149 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-30 13:51:25,163 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-30 13:51:25,257 [denest.simulation] INFO:     Calling `nest.SetKernelStatus

2020-06-30 13:51:26,754 [denest.session] INFO: Session '01_3_spikes' virtual running time: 100 ms
2020-06-30 13:51:26,755 [denest.session] INFO: Session '01_3_spikes' real running time: 0h:00m:00s
2020-06-30 13:51:26,758 [denest.simulation] INFO: Done running session '01_3_spikes'
2020-06-30 13:51:26,762 [denest.simulation] INFO: Running session: '02_2_spikes'...
2020-06-30 13:51:26,764 [denest.session] INFO: Initializing session...
2020-06-30 13:51:26,770 [denest.session] INFO: Setting `origin` flag to `200.0` for all stimulation devices in ``InputLayers`` for session `02_2_spikes`
2020-06-30 13:51:26,815 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', 'from_array': False}
2020-06-30 13:51:26,816 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times', from single value')
2020-06-30 13:51:26,885 [denest.session] INFO: Fini

### Run from the command line

A simulation can also be run directly from the command line as follows:

``python3 -m denest <tree_paths.yml> [-o <output_dir>]``